In [ ]:
# A bit of setup
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Create color maps
cmap_list = ['orange', 'cyan', 'cornflowerblue']
cmap_bold = ['darkorange', 'c', 'darkblue']
cmap_light = ListedColormap(cmap_list)

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

<!-- dom:TITLE: Homework 1, PHY 959 -->
<!-- dom:AUTHOR: [PHY 959: Machine Learning in Physics]-->


# PHY 959: Homework Set #2
Due: **February 9, 2021**



___
***


# Problem 1: Exploring Regularization

In this problem, we will exercise what we learned about regularization.  We will use two different data sets that allow us to see regularization at work with a linear regression problem.  You will be fitting a polynomial function to data that is mostly linear, but has enough noise to confuse our choice of polynomial function.  Because we don't technically know the highest order of the polynomial in the data, we need to allow for a wide range.  But if we use a high enough order, we'll easily overfit our data and lose the ability to predict new data.  The goal will be to learn to identify cases where we've overfit our data and to use regularization to reduce the overfitting.

There is a lot of "starter" code here that will not work out of the box.  You'll need to add your own code to fill in some gaps that are missing.  Ask Prof. Fisher if you get stuck.

## Part 1a:

In the next cell, you are given:

  1. A training data sample to use to fit your polynomial
  2. An independent testing data sample to evaluate the quality of your fit

Make a plot of these two samples.

In [ ]:
# Build and visualize the data

# Make some training data
nPts = 10
np.random.seed(119)
Xtrain = np.random.uniform(0,1,nPts)
ytrain = 0.1*Xtrain + 1*np.sin(Xtrain*15) 

# Make some testing data, 2x as many as training
np.random.seed(121)
Xtest = np.random.uniform(0,1,nPts*2)
ytest = 0.1*Xtest + 1*np.sin(Xtest*15)

# Because we're going to be plotting lines, it will be very convenient if
# our data is ordered by the absyssa.  We can use the argsort function to create a permutation.
print("Before permutation\n",Xtrain)
permute = Xtrain.argsort()
Xtrain=Xtrain[permute]
ytrain=ytrain[permute]
print("After permutation\n",Xtrain)

permute = Xtest.argsort()
Xtest=Xtest[permute]
ytest=ytest[permute]



## Part 1b:

Create a design/feature matrix for your samples.  In previous examples, the design matrix contained one variable for each dimension. Because we're now doing polynomial regression in one dimension, you will need to create a d-dimensional design matrix where d is the highest polynomial order.  For this problem, use d=7.  You can try out other values, but when you turn in your work please use 7 degrees of freedom.  Note that one of these will be the offset (bias) variable.

I strongly encourage you to try out the module ```PolynomialFeatures``` from ```scikit-learn```:

```
from sklearn.preprocessing import PolynomialFeatures

xt = np.zeros((X.size,1))
xt[:,0] = Xtrain
poly = PolynomialFeatures(degree=nDegr)
DM = poly.fit_transform(xt)  #this is your design matrix!
```

In this example code, note that the variable to be transformed needs to be an array of size ```(Npts,1)```.  Print out your design matrix so you can see what the ```PolynomialFeatures``` module does for you.  Don't forget to build a design matrix for your testing data!

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

nDegr = 7

## Part 1c:

Using your skills learned from homework #1, use the Numpy ```linalg``` module to calculate the vector of weights that best fit your data.  Make a plot of your training data and your best fit line.  A convenient way to generate your line would be like this:  ```line = np.dot(DM,betas)``` wherein ```betas``` is the weights vector from your ```linalg``` solution.

In [ ]:
#calculate our weights
betas = 

#print out our betas
print(betas)

#make a line!
line = np.dot(DM,betas)

#plot the line!
plt.scatter(Xtrain,ytrain)
plt.plot(Xtrain,line,c="red")
plt.show()

## Part 1d:

Now let's set up to do a fit of our data and add regularization to the mix.  I've provided a starting point to speed you up.  Note that there is a "toggle" to switch between L2 and L1 regularization.  You'll need to work out the correct gradients for each to make the code work properly.

In the following, you'll be making some plots. You do not need to turn in plots using both regularization schemes.  It is enough that the plots can be generated as you toggle between L2 and L1 regularization.

In [ ]:
#Linear regression

# Initialize parameters from our fit.  To reduce training time, we may as well
# use the results from our previous step! 
W = betas*0.99

# some hyperparameters
step_size = 0.1  # Note that if the learning rate is too big, you may not converge!
reg = 0.5 # regularization strength

#Toggle between L2 and L1 regularization
doL2 = 0

Nsteps = 10  # number of steps for the regularization strength to be tested over
rMin = 0.0001
rMax = 5

# toggle L1 and L2 regularization
if doL2:
    rMax = 1

# this array will hold the values of the regularization strength that we'll test
regRange = np.arange(rMin,rMax,(rMax-rMin)/Nsteps)

# this array will hold the results of our various fits
weights = np.zeros((regRange.size,nDegr+1))


# gradient descent loop
Niter = 250
for ridx, reg in enumerate(regRange):
    #always reset the weights between regularization cycles!
    W = betas*0.999
    
    for i in range(Niter):
  
        # evaluate function values
        fhat = np.dot(DM,W)

        # compute mean squared loss
        data_loss = np.sum((ytrain-fhat)**2)
        data_loss /= nPts

        # Regularize!
        # L2 regularization
        if doL2 :
            reg_loss = 
        else:
            reg_loss = 

        # Total loss is the sum
        loss = reg_loss + data_loss

        # Report progress
        if i % 100 == 0:
            print("Reg %.3f, iteration %d: loss %f" % (reg,i, loss))
        
        #compute the loss gradients
        dW = -2*np.dot(DM.T,ytrain-fhat)
        dW /= nPts
        
        if doL2:
            dW +=  # regularization gradient for L2
        else:            
            dW +=  # regularization gradient for L1
        
        #update weights after this iteration
        W -= step_size*dW

    #capture weights
    weights[ridx,:] = W
    print("Added: ", ridx)
    
print(weights)

## Part 1e:

You now have a weights matrix that is of size ```(regRange.size,nDegr+1)```, wherein each row is the weights found for a given regularization strength.  Make two figures using this data:

  1. Plot the value of each of your ```nDegr``` parameters (ordinate) as a function of the regularization strength (absyssa)
  2. Plot the resulting function found for each regularization strength value.  Include the training data in the figure.
  
Please use your figures to support answers to the following questions:

  1. What do you conclude about your choice of regularization scheme, L2 vs L1?
  2. At this point, can you state which regularization strength is "best" for each case?


## Part 1f:

Last one!!  To better understand the effect of regularization on this problem, let's study the testing data sample.  Make a final plot that shows the mean squared error loss as a function of regularization strength.  Note that you should be able to reuse a lot of the code from the previous step.  Be careful about finding the mean value, as your testing and training data samples are not the same size!

Use this figure and the previous two to answer the following:

  1. Which regularization strength appears to be best for both L1 and L2 regularization?
  2. For this problem, which regularization scheme appears to work better?  Why?